In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import itertools, os

In [ ]:

def generate_distinct_colors(n):
    colors = plt.cm.tab20.colors
    hex_colors = [matplotlib.colors.rgb2hex(color) for color in colors]
    
    # If n is greater than 20, repeat the colors
    hex_colors = hex_colors * (n // 20) + hex_colors[:n % 20]
    
    return hex_colors[:n]

def generate_distinct_linestyles(n):
    linestyles = ['-', '--', '-.', ':']
    linestyles = linestyles * (n // len(linestyles)) + linestyles[:n % len(linestyles)]
    return linestyles[:n]

def generate_distinct_markers(n):
    markers = ['o', 's', 'v', '^', '<', '>', 'd', 'p', '*', 'h', 'H', '+', 'x', 'D', '|', '_']
    markers = markers * (n // len(markers)) + markers[:n % len(markers)]
    return markers[:n]

def hex_to_rgb(hex_code):
    hex_code = hex_code.lstrip('#')
    return tuple(int(hex_code[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb_tuple):
    return '#{:02x}{:02x}{:02x}'.format(*rgb_tuple)

def combine_colors(hex_list):
    if not hex_list:
        return None
    
    total_rgb = [0, 0, 0]
    num_colors = len(hex_list)
    
    for hex_code in hex_list:
        rgb = hex_to_rgb(hex_code)
        total_rgb = [sum(x) for x in zip(total_rgb, rgb)]
    
    averaged_rgb = tuple(val // num_colors for val in total_rgb)
    return rgb_to_hex(averaged_rgb)

In [ ]:

def make_plots(to_plot_params):
    
    time_horizon = 100
    keys = to_plot_params.keys()
    values = to_plot_params.values()

    # Create all combinations of the parameter values
    product = itertools.product(*values)

    # For each combination, create a dictionary where the keys are mapped to the respective values
    labels_to_plot = [dict(zip(keys, combination)) for combination in product]

    n = len(labels_to_plot)
    colors = generate_distinct_colors(n)
    linestyles = generate_distinct_linestyles(n)
    markers = generate_distinct_markers(n)



    basic_info = ""
    shared_params = []

    for key in to_plot_params.keys():
        if len(to_plot_params[key]) == 1:
            shared_params.append(key)
            basic_info += f"{key}={to_plot_params[key][0]}-"
    basic_info = basic_info[:-1]

    for k in range(len(labels_to_plot)):

        labelstr = ""
        for key in to_plot_params.keys():
            if key not in shared_params:
                labelstr += f"{key}={labels_to_plot[k][key]}-"
        labelstr = labelstr[:-1]
        labels_to_plot[k]["label"] = labelstr
        labels_to_plot[k]["color"] = colors[k]
        labels_to_plot[k]["marker"] = markers[k]
        labels_to_plot[k]["linestyle"] = linestyles[k]



    for d in labels_to_plot:
        dp_threshold_str = f"{d['dp_threshold']:.4f}".split('.')[1]
        clean_filename = f"{d['dataset']}_{d['sensitive_attr']}_{d['ml_algo']}_{time_horizon}_{d['cost_type']}_{d['n_cost_bin']}_{dp_threshold_str}_{d['lambda_decision']}_"
        os.listdir("experimental_results/simulation_results/")
        data_path_list = [k for k in os.listdir("experimental_results/simulation_results/") if clean_filename in k]
        if len(data_path_list) == 0:
            raise Exception(f"No data found for {d}")
        for i in range(len(data_path_list)):
            data_path = f"experimental_results/simulation_results/{data_path_list[i]}"
            tmp_df = pd.read_csv(data_path, index_col = 0)
            tmp_df["acc_cost"] = tmp_df["cost"].cumsum()
            tmp_df["acc_util"] = tmp_df["utility"].cumsum()
            tmp_df['step'] = tmp_df.index
            tmp_df['sim_num'] = i
            if i == 0:
                df = tmp_df.copy()
            else:
                df = pd.concat([df, tmp_df], ignore_index = True)
            metrics_to_plot = ['acc_cost', 'acc_util', 'dp']

            for metric in metrics_to_plot:
                d[metric] = {}
                d[metric]["med"] = df[['step', metric]].groupby('step').quantile(0.50)[metric].values
    #             d[metric]["med"] = df[['step', metric]].groupby('step').mean(0)[metric].values
                d[metric]["min"] = df[['step', metric]].groupby('step').quantile(0.25)[metric].values
                d[metric]["max"] = df[['step', metric]].groupby('step').quantile(0.75)[metric].values
            # these are stated as med: medium, min, max for plot. They could be mean+-std as well (see commented out)
        steps = tmp_df['step'].unique()
        
        
    metrics_to_plot = ["dp", "acc_cost", "acc_util"]
    linewidth = 2

    for metric in metrics_to_plot:

        # Create the plot
        fig, ax = plt.subplots(figsize=(9,5))

        # plt.figure(figsize=(10, 6))

        for d in labels_to_plot:
            plt.plot(steps, d[metric]["med"], 
                     color = d["color"], marker = d["marker"], 
                     linestyle = d["linestyle"], linewidth = linewidth, label = d["label"])
            plt.fill_between(steps, d[metric]["min"], d[metric]["max"],
                            color = d["color"], alpha = 0.2)



        for spine in ['right', 'top']:
            ax.spines[spine].set_visible(False)

        # Add labels and title
        if metric == "dp":
            dp_targets = set()
            for d in labels_to_plot:
                dp_targets.add(d["dp_threshold"])
            for dp_target in dp_targets:
                colorlist = []
                for d in labels_to_plot:
                    colorlist.append(d["color"])
                color = combine_colors(colorlist)

                plt.axhline(dp_target, linestyle='dashed', label="Target DP",
                            xmin = 0.05, xmax = 0.96, color = color)

        plt.xlabel('Time steps')
        plt.ylabel(metric)
        plot_title = ", ".join(basic_info.split("-"))
        plt.title(plot_title)

        plt.legend()


        # Show the plot
        plt.savefig(f'experimental_results/graphs/{metric}_{plot_title}.pdf', bbox_inches='tight')
        plt.show()

In [ ]:

to_plot_params = {
    "dataset": ["adult"],
    "sensitive_attr" :  ["race"],
    "ml_algo" : ["erm", "diffdp"],
    "cost_type" : ["constant"],
    "n_cost_bin" : [10],
    "lambda_decision" : [1],
    "dp_threshold" : [0.1]
}

make_plots(to_plot_params)